Lambda School Data Science

*Unit 2, Sprint 2, Module 1*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?

### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [3]:
# Import

%matplotlib inline

import pandas_profiling

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

from sklearn.pipeline import make_pipeline
import category_encoders as ce
from scipy.stats import randint, uniform

In [4]:
from sklearn.model_selection import train_test_split

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [5]:
import numpy as np

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    # return the wrangled dataframe
    return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [6]:
# The status_group column is the target
target = 'status_group'
features = train.columns.drop(target)

In [7]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test

In [8]:
from sklearn.ensemble import RandomForestClassifier

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names = True, cols = ['basin']),
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state = 42),
)

In [9]:
from scipy.stats import randint, uniform

#Hyperparameter optimization with RandomSearchCV

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__n_estimators': randint(50, 500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1),
}

In [10]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score


# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=5, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 20.6min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 28.5min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 37.9min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 47.0min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 62.1min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 74.1min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 81.0min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 93.0min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 109.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 122.9min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 

In [11]:
print('Best H yperparameters:', search.best_params_)
print('Accuracy:', search.best_score_)
print('Best Estimator:', search.best_estimator_)

Best H yperparameters: {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.14914080866979118, 'randomforestclassifier__n_estimators': 446, 'simpleimputer__strategy': 'mean'}
Accuracy: 0.8087121212121212
Best Estimator: Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['basin'], drop_invariant=False,
                               handle_missing='value', handle_unknown='value',
                               return_df=True, use_cat_names=True, verbose=0)),
                ('ordinalencoder',
                 OrdinalEncoder(cols=['funder', 'installer', 'wpt_name',
                                      'subvillage', 'region', 'lga', 'ward',
                                      'public_meeting', 'scheme_management',
                                      'scheme_name', 'permi...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_dep

In [12]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,27,18,76,93,5,29,33,11,44,23,...,3,35,8,96,70,61,66,13,68,14
mean_fit_time,180.262,63.5658,99.8349,105.769,78.2111,95.596,125.201,170.241,29.9867,162.681,...,28.3608,36.8104,185.64,9.48814,51.732,108.117,101.127,211.745,101.011,14.8882
std_fit_time,1.4141,3.4535,2.58708,1.34594,19.6381,14.6146,1.81522,3.30061,0.430128,40.0603,...,1.16232,0.23328,40.525,3.36942,0.538123,1.75726,1.26326,1.99809,2.34637,0.0939645
mean_score_time,6.07628,2.24017,2.30782,2.15549,4.96506,1.70277,1.86559,2.8417,0.821778,3.42919,...,1.33523,0.678773,2.14753,0.814995,0.706643,0.990844,0.942328,1.96374,1.04023,1.16541
std_score_time,0.117447,0.608544,0.102947,0.15213,1.99744,0.0422765,0.0458563,0.0763345,0.0280747,0.898141,...,0.0305972,0.00645639,0.627153,0.213096,0.0386449,0.0346361,0.0135162,0.0075891,0.0572197,0.0250481
param_randomforestclassifier__max_depth,20,20,20,20,20,20,20,20,20,20,...,5,5,5,10,5,5,5,5,5,5
param_randomforestclassifier__max_features,0.149141,0.142597,0.221295,0.279732,0.080375,0.284027,0.37684,0.339925,0.31528,0.318747,...,0.204084,0.66854,0.838344,0.00929959,0.939939,0.9053,0.952105,0.961859,0.784073,0.12812
param_randomforestclassifier__n_estimators,446,315,448,442,327,353,408,277,104,248,...,167,181,277,109,187,419,379,395,454,100
param_simpleimputer__strategy,mean,mean,mean,median,mean,mean,mean,median,mean,mean,...,mean,median,mean,mean,median,mean,median,median,mean,mean
params,"{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...",...,"{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand..."
split0_test_score,0.809995,0.808522,0.809784,0.807365,0.809995,0.806628,0.805366,0.805576,0.803893,0.804208,...,0.725302,0.72425,0.723619,0.715308,0.723409,0.724356,0.722567,0.722462,0.724987,0.725513


In [14]:
assert all(X_train.columns == X_test.columns)

In [15]:
# Predict on test
pipeline = search.best_estimator_
y_pred = pipeline.predict(X_test)
y_pred.shape

(14358,)

In [16]:
#Write a submission csv file

submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('Obukwelu_DSPT5_Unit_2_submission5_1.csv', index=False)

In [17]:
submission.head()

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [18]:
submission.shape

(14358, 2)